# <center>Algorithmes pour le traitement de séquences.<br/>Alignement optimal et logiciel d’aide à la dététection de plagiat</center>
<center>https://perso.esiee.fr/~coustyj/Sequences/IT-4301E-TDm2.html</center>

Importons les packets utiles pour la suite et chargons le contenu des fichier en variable

In [295]:
import numpy as np
t1 = open('texte1.txt').read()
t2 = open('texte2.txt').read()

## Question 1
**Proposer un algorithme en O(|x| × |y|) pour calculer le score d’un alignement optimal entre x et y (indications : on essaiera de se ramener à un problème connu).**

Afin de calculer le score d'alignement optimal entre deux string, ici t1 et t2, nous allons calculer leur distance de Levenshtein

## Question 2

**On considère la matrice T de taille (|x|+1) × (|y|+1) telle que T[i][j] est le score d’un alignement optimal entre xi et yj, où xi et yj désignent les préfixes de x et de y de longueur i et j respectivement.**

**Étant donnée la matrice T (obtenue par exemple avec l’algorithme de la question 1) et les séquences x et y, proposez un schéma de programme retournant un alignement optimal (x′,y′) de x et y.**

**Évaluer la complexité de calcul de l’algorithme proposé; si celle-ci n’est pas linéaire, proposez un autre algorithme de complexité linéaire.**

In [2]:
def levenshtein_distance(a :str, b:str) -> int:
    n,m = len(a),len(b)
    T = np.zeros((n+1,m+1))
    T[:,0] = range(n+1)
    T[0,:] = range(m+1)
    
    for i in range(n):
        for j in range(m):
            same = int(a[i]!=b[j])
            T[i+1,j+1] = min(
                        T[i, j] + same, # NOTE: Substitution
                        T[i, j+1] + 1, # NOTE: Insertion
                        T[i+1, j] + 1  # NOTE: Delete
                    )
            
    return T

# Question 3

Implémentez les algorithmes des questions 1 et 2 et testez les en affichant un alignement optimal de texte1.c et texte2.c

In [3]:
res = levenshtein_distance(t1,t2)

In [9]:
def reverse_lev(T, i, j, x, y):
    if i==0:
        return ['-%s'%y[i] for i in range(j)]
    if j==0:
        return ['+%s'%x[j] for j in range(i)]
    xi, yj = x[i-1], y[j-1]

    if xi == yj:
        return reverse_lev(T, i-1, j-1, x, y) + ['\\']
    
    else:
        
        if T[i-1, j-1] < T[i, j]:
            return reverse_lev(T, i-1, j-1, x, y)+ ['\\%s\t'%xi]

        elif T[i-1, j] < T[i, j]:
            return reverse_lev(T, i-1, j, x, y) + ['+%s'%yj]
        
        else:
            return reverse_lev(T, i, j-1, x, y) + ['-%s'%yj]
        

In [6]:
levenshtein_distance(open('texte1.txt').read(),open('texte2.txt').read())[:,-1]

array([1801., 1800., 1799., ...,  579.,  578.,  577.])

In [7]:
levenshtein_distance('a','b')

array([[0., 1.],
       [1., 1.]])

In [47]:
tt= []
for a,b in zip(t1.split('\n'),t2.split('\n')):
    lev = levenshtein_distance(a,b)[-1,-1]
#     print(lev)
    tt.append(lev)
print(sum(tt))

789.0


In [297]:
T1 = "j'ai écris un texte de ouf"
T2 = "kk kk k k kkkk  kk k kk moi aussi j'ai écris un texte de ouf"
T = levenshtein_distance(T1,T2)

print('Pour transformer T2 en T1 \n')
print(reverse_lev(T,T.shape[0]-1,T.shape[1]-1,list(T1),list(T2)))

Pour transformer T2 en T1 

__________________________________j'ai écris un texte de ouf
kk kk k k kkkk  kk k kk moi aussi j'ai écris un texte de ouf


In [283]:
def reverse_lev(T, i, j, x, y):
    
    if i==0:
        x = ['_']*j + x
        return '%s\n%s'%(''.join(x), ''.join(y))
    if j==0:
        y = ['_']*i + y
        return '%s\n%s'%(''.join(x), ''.join(y))
    
    xi, yj = x[i-1], y[j-1]

    if xi == yj:
        return reverse_lev(T, i-1, j-1, x, y)
    
    else:
        
        if T[i-1, j-1] < T[i, j]:
            return reverse_lev(T, i-1, j-1, x, y)

        elif T[i-1, j] < T[i, j]:
            y.insert(j,'_')
            return reverse_lev(T, i-1, j, x, y)
        
        else:
            x.insert(i,'_')
            return reverse_lev(T, i, j-1, x, y)